## Imports

In [ ]:
#Gabriel/Piotrek: Choose index from 1 - 8 for variable PREPROCESSING_OPTIONS[INDEX HERE]
#Run the notebook from top to bottom
#Save stats; rename it if needed

#In this cell, there are various option for the user to freely choose:
#embedding_choice
#PREPROCESSING_CHOICE

import time
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import gensim.downloader
import pandas as pd
import numpy as np
import torch
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

from gensim.models import Word2Vec
from gensim.models import FastText
from gensim.models import KeyedVectors
#from gensim.test.utils import common_texts
import gensim.downloader as api
import gensim

#helper function to save models and stats
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
import pickle


use_drive = True
if use_drive:
  PATH = "/content/drive/MyDrive/CIL 2022/"
  from google.colab import drive
  drive.mount('/content/drive')
  %cd /content/drive/MyDrive/CIL 2022/
  !ls
else:
  PATH = "./"

#save model
# now you can save it to a file
# with open('filename.pkl', 'wb') as f:
#     pickle.dump(clf, f)

# # and later you can load it
# with open('filename.pkl', 'rb') as f:
#     clf = pickle.load(f)

#1 = stanford glove
#2 = word2vec
#3 = fasttext
#4 = basic urban dictionary embedding
#5 = phrase urban dictionary embedding
def get_embedding_name(embedding_id):
  if embedding_id == 1:
    return "stanford glove"
  elif embedding_id == 2:
    return "word2vec"
  elif embedding_id == 3:
    return "fasttext"
  elif embedding_id == 4:
    return "basic urban dictionary"
  elif embedding_id == 5:
    return "phrase urban dictionary"
  else:
    return "User error: embedding id does not exist."

#Saving trained model
def save_model(model, model_name):
  name_of_file_model = "trained " + model_name + ".pkl"
  with open(name_of_file_model, 'wb') as f:
    pickle.dump(model, f)
  print("Model saved.")

#Saving stats of model to a file
def save_stats_to_file(y_val, y_val_pred, model_name):
  name_of_file_stats = get_embedding_name(embedding_choice) + "+" + model_name + PREPROCESSING_CHOICE + ".txt"
  # name_of_file_model = "trained " + model_name + ".pkl"
  # print("Saving model to file.")
  # with open(name_of_file_model, 'wb') as f:
  #   pickle.dump(model, f)
  print("Saving model relevant stats to file.")
  file_obj = open(name_of_file_stats, "w", encoding="utf8")
  file_obj.write(f'use_drive: {use_drive}\n')
  # file_obj.write(f'LOAD_FROM_SPLITTED_DATASET: {LOAD_FROM_SPLITTED_DATASET}\n')
  # file_obj.write(f'WRITE_SPLITTED_DATASET: {WRITE_SPLITTED_DATASET}\n')
  # file_obj.write(f'REMOVE_DUPLICATES_FROM_TRAINING: {REMOVE_DUPLICATES_FROM_TRAINING}\n')
  file_obj.write(f'PREPROCESSING_CHOICE: {PREPROCESSING_CHOICE}\n')
  file_obj.write(f'embedding_choice: {embedding_choice}\n')
  file_obj.write(f'embedding_name: {get_embedding_name(embedding_choice)}\n')
  file_obj.write(f'dimension_of_embedding: {dimension_of_embedding}\n')
  file_obj.write(f'Model name: {model_name}\n')
  file_obj.write(f'Acc: {accuracy_score(y_val, y_val_pred)}\n')
  file_obj.write(f'Recall: {recall_score(y_val, y_val_pred)}\n')
  file_obj.write(f'Precision: {precision_score(y_val, y_val_pred)}\n')
  file_obj.write(f'F1: {f1_score(y_val, y_val_pred)}\n')
  file_obj.write(f'ROC_AUC: {roc_auc_score(y_val, y_val_pred)}\n')
  file_obj.close()
  print(f'Acc: {accuracy_score(y_val, y_val_pred)}')
  print(f'Recall: {recall_score(y_val, y_val_pred)}')
  print(f'Precision: {precision_score(y_val, y_val_pred)}')
  print(f'F1: {f1_score(y_val, y_val_pred)}')
  print(f'ROC_AUC: {roc_auc_score(y_val, y_val_pred)}')


#Saving stats of model to a file
def save_epoch_to_file(epoch, valid_loss, valid_acc, model_name):
  name_of_file_stats = get_embedding_name(embedding_choice) + "+" + model_name + PREPROCESSING_CHOICE + ".txt"

  print("Saving model relevant stats to file.")
  file_obj = open(name_of_file_stats, "a", encoding="utf8")
  file_obj.write(f'Epoch number: {epoch}\n')
  file_obj.write(f'PREPROCESSING_CHOICE: {PREPROCESSING_CHOICE}\n')
  file_obj.write(f'embedding_choice: {embedding_choice}\n')
  file_obj.write(f'embedding_name: {get_embedding_name(embedding_choice)}\n')
  file_obj.write(f'dimension_of_embedding: {dimension_of_embedding}\n')
  file_obj.write(f'Model name: {model_name}\n')
  file_obj.write(f'Acc: {valid_acc*100:.2f}%\n')
  file_obj.write(f'Loss: {valid_loss:.3f}\n')
  file_obj.close()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1NSBJKoA0FA5BeDWR8bNo5tPgA5lCU8yv/CIL 2022
'code submission README.txt'
 data
'Final Results.gsheet'
 Models
 OLD
 Preprocessing
'README document.gdoc'
'Saved Model States'
'stanford glove+GRU_stanford glove_no-stemming_no-lemmatize_with-stopwords_no-spellcorrectno-stemming_no-lemmatize_with-stopwords_no-spellcorrect.txt'
'trained GRU_stanford glove_no-stemming_no-lemmatize_with-stopwords_no-spellcorrect.pkl'
'Word embeddings'


# Choices

In [ ]:
#Choose embedding below
#1 = stanford glove
#2 = word2vec
#3 = fasttext
#4 = basic urban dictionary embedding
#5 = phrase urban dictionary embedding
embedding_choice = 4

USE_DICTIONARY = True
embedding_non_dict_model = None
embedding_dict = dict()
#SET FLAG "USE_DICTIONARY" to determine if we work with a dictionary or a loaded pretrained model


# HYPERPARAMETERS
# LOAD_FROM_SPLITTED_DATASET = True
# WRITE_SPLITTED_DATASET = True
# REMOVE_DUPLICATES_FROM_TRAINING = False

#option 0 - 8
PREPROCESSING_OPTIONS = [ "raw",
"no-stemming_no-lemmatize_no-stopwords_no-spellcorrect",
"no-stemming_no-lemmatize_with-stopwords_no-spellcorrect",
"no-stemming_no-lemmatize_with-stopwords_with-spellcorrect",
"no-stemming_with-lemmatize_with-stopwords_no-spellcorrect",
"no-stemming_with-lemmatize_with-stopwords_with-spellcorrect",
"with-stemming_no-lemmatize_with-stopwords_no-spellcorrect",
"with-stemming_with-lemmatize_no-stopwords_with-spellcorrect",
"with-stemming_with-lemmatize_with-stopwords_no-spellcorrect" ]
PREPROCESSING_CHOICE = PREPROCESSING_OPTIONS[0] # one from PREPROCESSING_OPTIONS

DEVICE = 'cuda'
N_EPOCHS = 30


model_name = "LSTM_" + get_embedding_name(embedding_choice) + "_" + PREPROCESSING_CHOICE

name_of_file_model = "trained " + model_name + ".pkl"


#Choose embedding

In [ ]:
#From the glove embedding file

#https://github.com/RaRe-Technologies/gensim-data#datasets
#https://kavita-ganesan.com/easily-access-pre-trained-word-embeddings-with-gensim/#.YsGEXexBxE4

#depending on choice, we choose a different file
if embedding_choice == 1:
  #stanford
  #files offers dimension_of_embedding = {25,50,100,200}
  dimension_of_embedding = 200 #can also modify this

  #easy:
  # model_name = "glove-twitter-" + str(dimension_of_embedding)
  # model = api.load(model_name)

  #directly from stanford glove file:
  path_to_embedding_file = "/content/drive/MyDrive/CIL 2022/Word embeddings"
  path_to_embedding_file += "/Glove/glove.twitter.27B." + str(dimension_of_embedding) + "d.txt"

  # FLAG_shaped_already_saved = False
  # shape_of_np_vector = None

  embedding_file_obj = open(path_to_embedding_file, "r", encoding = "utf8")
  for line in embedding_file_obj:
    #Check if line consists only of ASCII characters (no Chinese/Japanse/Arabic, etc. expressions)
    #There were some lines that had non-ASCII characters.
    if line.isascii():
      word_n_vector = line.split()
      embedding_dict[word_n_vector[0]] = np.array(word_n_vector[1:], dtype=np.float64)
      # if not FLAG_shaped_already_saved:
      #   shape_of_np_vector = np.array(word_n_vector[1:], dtype=np.float64).shape
      #   FLAG_shaped_already_saved = True

  embedding_file_obj.close

  #add <UNKNOWN> token to vocabulary
  #choose the 0 vector as the corresponding embedding
  embedding_dict["<UNKNOWN>"] = np.zeros(dimension_of_embedding, dtype=np.float64)
  embedding_dict["<PAD>"] = np.zeros(dimension_of_embedding, dtype=np.float64)
  #embedding_dict["<UNKNOWN>"] = np.zeros(shape_of_np_vector, dtype=np.float64)


elif embedding_choice == 2:
  
#https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html

  dimension_of_embedding = 300
  #choose which embeddings
  # model_name = "word2vec-google-news-300"
  # model_name = "word2vec-ruscorpora-300" #get Russian vocabulary

  #directly from word2vec file:
  path_to_embedding_file = "/content/drive/MyDrive/CIL 2022/Word embeddings"
  path_to_embedding_file += "/word2vec/GoogleNews-vectors-negative300.txt"


  embedding_file_obj = open(path_to_embedding_file, "r", encoding = "utf8")
  for line in embedding_file_obj:
    #Check if line consists only of ASCII characters (no Chinese/Japanse/Arabic, etc. expressions)
    #There were some lines that had non-ASCII characters.
    if line.isascii():
      word_n_vector = line.split()
      embedding_dict[word_n_vector[0]] = np.array(word_n_vector[1:], dtype=np.float64)


  embedding_file_obj.close

  #add <UNKNOWN> token to vocabulary
  #choose the 0 vector as the corresponding embedding
  embedding_dict["<UNKNOWN>"] = np.zeros(dimension_of_embedding, dtype=np.float64)
  embedding_dict["<PAD>"] = np.zeros(dimension_of_embedding, dtype=np.float64)

elif embedding_choice == 3:

#https://radimrehurek.com/gensim/auto_examples/tutorials/run_fasttext.html#sphx-glr-auto-examples-tutorials-run-fasttext-py
  
  dimension_of_embedding = 300
  #choose which embeddings
  # model_name = "word2vec-google-news-300"
  # model_name = "word2vec-ruscorpora-300" #get Russian vocabulary

  #directly from word2vec file:
  path_to_embedding_file = "/content/drive/MyDrive/CIL 2022/Word embeddings"
  path_to_embedding_file += "/fasttext/wiki-news-300d-1M-subword.vec"


  embedding_file_obj = open(path_to_embedding_file, "r", encoding = "utf8")
  for line in embedding_file_obj:
    #Check if line consists only of ASCII characters (no Chinese/Japanse/Arabic, etc. expressions)
    #There were some lines that had non-ASCII characters.
    if line.isascii():
      word_n_vector = line.split()
      embedding_dict[word_n_vector[0]] = np.array(word_n_vector[1:], dtype=np.float64)


  embedding_file_obj.close

  #add <UNKNOWN> token to vocabulary
  #choose the 0 vector as the corresponding embedding
  embedding_dict["<UNKNOWN>"] = np.zeros(dimension_of_embedding, dtype=np.float64)
  embedding_dict["<PAD>"] = np.zeros(dimension_of_embedding, dtype=np.float64)
  

elif (embedding_choice == 4) or (embedding_choice == 5):
  
  #see https://aclanthology.org/2020.lrec-1.586.pdf for difference.

  #TODO: figure out what the binaries do
  path_to_embedding_file = None
  if embedding_choice == 4:
    path_to_embedding_file = "/content/drive/MyDrive/CIL 2022/Word embeddings/urbanDictionary/ud_basic.vec"
  else:
    path_to_embedding_file = "/content/drive/MyDrive/CIL 2022/Word embeddings/urbanDictionary/ud_phrase.vec"
  

  embedding_file_obj = open(path_to_embedding_file, "r")
  vocab_size_and_embedding_dim = embedding_file_obj.readline().split()

  vocab_size = int(vocab_size_and_embedding_dim[0])
  dimension_of_embedding = int(vocab_size_and_embedding_dim[1])

  # FLAG_shaped_already_saved = False
  # shape_of_np_vector = None

  for line in embedding_file_obj:
    word_n_vector = line.split()
    embedding_dict[word_n_vector[0]] = np.array(word_n_vector[1:], dtype=np.float64)
    # if not FLAG_shaped_already_saved:
    #   shape_of_np_vector = np.array(word_n_vector[1:], dtype=np.float64).shape
    #   FLAG_shaped_already_saved = True

  embedding_file_obj.close

  #add <UNKNOWN> token to vocabulary
  #choose the 0 vector as the corresponding embedding
  embedding_dict["<UNKNOWN>"] = np.zeros(dimension_of_embedding, dtype=np.float64)
  embedding_dict["<PAD>"] = np.zeros(dimension_of_embedding, dtype=np.float64)
  #embedding_dict["<UNKNOWN>"] = np.zeros(shape_of_np_vector, dtype=np.float64)
else:
  print("Error: embedding not recognized")

#init method

In [ ]:
print(embedding_dict["<PAD>"])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


# Tokenize to embeddings

In [ ]:

#https://www.nltk.org/api/nltk.tokenize.html

#feature = average of vectors of tokens occuring in tweet
def transform_to_embeddings(list_of_sentences, embedding_model, embedding_model_type, word_to_ix):

  max_size = 150
  sentences = torch.zeros(len(list_of_sentences), max_size, dtype=torch.long, device=DEVICE)#to return
  lengths = torch.zeros(len(list_of_sentences), dtype=torch.long, device=DEVICE)
    #for each tweet:
    #for each token:
    #get embedding vector
    #compute average
    #store it in a list with corresponding label
  pad_index = word_to_ix.get('<PAD>')

  for line, tweet in enumerate(list_of_sentences):
    tokenized_tweet = tknzr.tokenize(tweet)
    indexed_embeddings = torch.tensor([pad_index]*max_size, dtype=torch.long, device=DEVICE)
    sentence_length = 0
    for index, word in enumerate(tokenized_tweet):
  
      word_index = word_to_ix.get(word)
      if word_index == None:
        word_index = word_to_ix.get("<UNKNOWN>")

      indexed_embeddings[index] = word_index
      sentence_length = sentence_length + 1
    sentences[line] = indexed_embeddings
    lengths[line] = sentence_length
  return sentences, lengths


#Create training and test set

In [ ]:
def read_file_and_strip(filename):
  lines = []
  with open(filename) as file:
    for line in file:
      lines.append(line.strip())
  return np.array(lines)

def read_data():
  dataset_path = PATH + "data/" + PREPROCESSING_CHOICE + "/"

  train_sentences = read_file_and_strip(dataset_path + "train_sentences.txt")
  train_labels = read_file_and_strip(dataset_path + "train_labels.txt").astype(int)
  val_sentences = read_file_and_strip(dataset_path + "val_sentences.txt")
  val_labels = read_file_and_strip(dataset_path + "val_labels.txt").astype(int)
  
  return train_sentences, train_labels, val_sentences, val_labels

train_sentences, train_labels, val_sentences, val_labels = read_data()
print(len(train_sentences))


embedding_model = None
if USE_DICTIONARY:
  embedding_model_type = "DICT"
  embedding_model = embedding_dict
else:
  embedding_model_type = "NON-DICT"
  embedding_model = embedding_non_dict_model


1970002


## Transform the sentences into tensor vectors of word indexes from the vocab corpus

In [ ]:
# Create a vocabulary corpus

word_to_ix = {word: i for i, word in enumerate(embedding_model)}

train_data, train_lengths = transform_to_embeddings(train_sentences, embedding_model, embedding_model_type, word_to_ix)
del train_sentences
val_data, val_lengths = transform_to_embeddings(val_sentences, embedding_model, embedding_model_type, word_to_ix)
del val_sentences
del embedding_dict

In [ ]:

train_labels = torch.tensor(train_labels, dtype=torch.long, device=DEVICE) 

val_labels = torch.tensor(val_labels, dtype=torch.long, device=DEVICE) 

## Transform the embeddings into float tensor

In [ ]:
# Transform the embeddings into float tensor with just weights and indexes of 
# those weights representing the words
test = np.zeros((len(embedding_model), dimension_of_embedding))

  
for i, word in enumerate(embedding_model):
    test[i] = embedding_model[word]
embedding_len = len(embedding_model)
del embedding_model


pretrained_embeddings = torch.tensor(test, dtype=torch.float, device=DEVICE)

#Model = RNN

## Put tensor into a dataset for batching

In [ ]:
class Dataset(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, list_id, labels, length):
        'Initialization'
        self.labels = labels
        self.list_id = list_id
        self.length = length

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_id)

  def __getitem__(self, index):
        'Generates one sample of data'

        # Load data and get label
        X = self.list_id[index]
        y = torch.tensor([self.labels[index]], dtype=torch.long, device=DEVICE)
        len = self.length[index]
        return X, y, len

In [ ]:
BATCH_SIZE = 512

dataset = Dataset(train_data, train_labels, train_lengths)
my_dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True) # create your dataloader
del train_data
del dataset
val_dataset = Dataset(val_data, val_labels, val_lengths)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
del val_data
del val_dataset

## Model

In [ ]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        # lengths need to be on CPU!
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to('cpu'), enforce_sorted=False)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)

## Training method

In [ ]:

from tqdm.auto import tqdm
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
      #Show progress bar to estimate time/epoch
    progress_bar = tqdm(range(len(iterator)))
    model.train()
    
    for index, batch in enumerate(iterator):
        optimizer.zero_grad()

        text, labels, lengths = batch
        #text = text.to(DEVICE)
        labels = labels.type(torch.FloatTensor).to(DEVICE)

        predictions = model(text.T, lengths)
        
        loss = criterion(predictions, labels)
        
        acc = binary_accuracy(predictions, labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        #advance progess ba by 1
        progress_bar.update(1)
          
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

## Evaluation method

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for index, batch in enumerate(iterator):
          text, labels, lengths = batch
          #text = text.to(DEVICE)
          labels = labels.type(torch.FloatTensor).to(DEVICE)

          predictions = model(text.T, lengths)
          
          loss = criterion(predictions, labels)
          
          acc = binary_accuracy(predictions, labels)
          

          epoch_loss += loss.item()
          epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

# Run Model

In [ ]:
pad_index = word_to_ix.get('<PAD>')
del word_to_ix

In [ ]:


INPUT_DIM = embedding_len
embedding_dim = dimension_of_embedding
hidden_dim = 256
output_dim = 1
n_layers = 3
bidirectional = True
dropout_rate = 0.5


model = RNN(INPUT_DIM, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout_rate, 
             pad_index)

model.embedding.from_pretrained(pretrained_embeddings, freeze=False, padding_idx=pad_index)


best_valid_loss = float('inf')


optimizer = optim.Adam(model.parameters())
#optimizer = optim.SGD(model.parameters(), lr=0.1)
criterion = nn.BCEWithLogitsLoss()


model = model.to(DEVICE)
criterion = criterion.to(DEVICE)


def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division #
    acc = correct.sum() / len(correct)
    return acc

for epoch in range(N_EPOCHS):


    start_time = time.time()
    
    train_loss, train_acc = train(model, my_dataloader, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, val_dataloader, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), name_of_file_model)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    save_epoch_to_file(epoch, valid_loss, valid_acc, model_name)

  0%|          | 0/3848 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 9m 54s
	Train Loss: 0.428 | Train Acc: 79.58%
	 Val. Loss: 0.382 |  Val. Acc: 82.49%
Saving model relevant stats to file.


  0%|          | 0/3848 [00:00<?, ?it/s]

Epoch: 02 | Epoch Time: 9m 54s
	Train Loss: 0.379 | Train Acc: 82.63%
	 Val. Loss: 0.364 |  Val. Acc: 83.70%
Saving model relevant stats to file.


  0%|          | 0/3848 [00:00<?, ?it/s]

...

  0%|          | 0/3848 [00:00<?, ?it/s]

Epoch: 26 | Epoch Time: 9m 51s
	Train Loss: 0.306 | Train Acc: 86.64%
	 Val. Loss: 0.350 |  Val. Acc: 85.15%
Saving model relevant stats to file.


  0%|          | 0/3848 [00:00<?, ?it/s]

Epoch: 27 | Epoch Time: 9m 51s
	Train Loss: 0.305 | Train Acc: 86.68%
	 Val. Loss: 0.351 |  Val. Acc: 85.11%
Saving model relevant stats to file.


  0%|          | 0/3848 [00:00<?, ?it/s]

In [ ]:
model.parameters